In [17]:
import umap, numpy as np, matplotlib.pyplot as plt, pandas as pd, seaborn as sns
from sklearn import cluster, datasets
from sklearn.metrics import silhouette_score as siluet
from sklearn.metrics.cluster import homogeneity_score as purity
from sklearn.metrics import normalized_mutual_info_score as NMI

In [18]:
# URL raw dari file CSV
url = 'https://raw.githubusercontent.com/Jepees/Data-Mining-Dani/refs/heads/main/CaseStudy-02/Data-Case-Study-02-MixITup.csv'

# Membaca data dari file CSV
df = pd.read_csv(url)

# Menampilkan beberapa baris pertama dari DataFrame
df.head()

,ID,Jenis Kelamin,Umur,Pendapatan-per-bulan,skor-pengeluaran,Menikah,Kota,Promo,HariFav,FavFlavor
0,101,Wanita,49,"Rp13,114,754",52,1,Jakarta,1.0,Sabtu,2
1,102,Wanita,32,"Rp33,606,557",86,1,Depok,1.0,Minggu,6
2,103,Pria,51,"Rp15,983,606",41,1,Jakarta,2.0,Minggu,3
3,104,Pria,19,"Rp12,704,918",55,0,Jakarta,1.0,Rabu,7
4,105,Wanita,38,"Rp20,081,967",42,1,Jakarta,2.0,Senin,8


In [19]:
print(f'shape: {df.shape}\nkolom: {df.columns}')

shape: (200, 10)
kolom: Index(['ID', 'Jenis Kelamin', 'Umur', 'Pendapatan-per-bulan',
       'skor-pengeluaran', 'Menikah', 'Kota', 'Promo', 'HariFav', 'FavFlavor'],
      dtype='object')


In [20]:
df.drop('ID', axis=1, inplace=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Jenis Kelamin         196 non-null    object 
 1   Umur                  200 non-null    object 
 2   Pendapatan-per-bulan  200 non-null    object 
 3   skor-pengeluaran      200 non-null    int64  
 4   Menikah               188 non-null    object 
 5   Kota                  200 non-null    object 
 6   Promo                 194 non-null    float64
 7   HariFav               200 non-null    object 
 8   FavFlavor             200 non-null    int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 14.2+ KB


In [22]:
for kolom in df.columns:
    print(f'kolom: {kolom}\ntotal unique: {df[kolom].nunique()}\nunique: {df[kolom].unique()}\n')

kolom: Jenis Kelamin
total unique: 3
unique: ['Wanita' 'Pria' nan '-']

kolom: Umur
total unique: 45
unique: ['49' '32' '51' '19' '38' '33' '27' '24' '21' '31' '36' '46' '60' '58'
 '47' '45' '22' '18' '59' '44' '39' '35' '20' '30' '23' '48' '37' '40'
 '34' '41' '53' '54' '28' '55' '25' '50' '42' '29' '43' '52' '2' "35'"
 '56' '26' '57']

kolom: Pendapatan-per-bulan
total unique: 64
unique: ['Rp13,114,754' 'Rp33,606,557' 'Rp15,983,606' 'Rp12,704,918'
 'Rp20,081,967' 'Rp29,098,360' 'Rp21,311,475' 'Rp2,049,180' 'Rp13,524,590'
 'Rp19,262,295' 'Rp11,475,409' 'Rp29,508,196' 'Rp34,016,393' 'Rp6,147,540'
 'Rp11,885,245' 'Rp25,819,672' 'Rp25,000,000' 'Rp5,327,868' 'Rp9,836,065'
 'Rp23,770,491' 'Rp22,950,819' 'Rp1,229,508' 'Rp9,016,393' 'Rp9,426,229'
 'Rp29,918,032' 'Rp28,688,524' 'Rp27,049,180' 'Rp18,442,622'
 'Rp19,672,131' 'Rp11,065,573' 'Rp36,065,573' 'Rp20,491,803'
 'Rp17,622,950' 'Rp409,836' 'Rp25,409,836' 'Rp1,639,344' 'Rp5,737,704'
 'Rp34,426,229' 'Rp10,245,901' 'Rp50,000,000' 'Rp14,344,

In [23]:
df.loc[df['Umur'] == "35'", 'Umur'] = 35
df = df[df['Umur'] != '2']
df.loc[df['Kota'] == "JJakarta", 'Kota'] = "Jakarta"
df.loc[df['Menikah'] == '-', 'Menikah'] = np.nan

In [24]:
df['Jenis Kelamin'] = df['Jenis Kelamin'].astype('category')
df['Umur'] = df['Umur'].astype(int)
df['Pendapatan-per-bulan'] = df['Pendapatan-per-bulan'].str.replace('Rp', '', regex=False)
df['Pendapatan-per-bulan'] = df['Pendapatan-per-bulan'].str.replace(',', '', regex=False)
df['Pendapatan-per-bulan'] = df['Pendapatan-per-bulan'].astype(int)

In [25]:
a = df[['Jenis Kelamin', 'Umur', 'Menikah']]
rerata_cwo = a[a['Jenis Kelamin'] == 'Pria']["Umur"].sum() // a[a['Jenis Kelamin'] == 'Pria']["Umur"].count()
rerata_cwe = a[a['Jenis Kelamin'] == 'Wanita']["Umur"].sum() // a[a['Jenis Kelamin'] == 'Wanita']["Umur"].count()
rerata_pendapatan = df['Pendapatan-per-bulan'].sum() // df['Pendapatan-per-bulan'].count()

print(rerata_cwo, rerata_cwe, rerata_pendapatan)

38 36 18675343


In [26]:
df.loc[(df['Menikah'].isna()) & (df['Jenis Kelamin'] == 'Pria') & (df['Umur'] >= rerata_cwo), 'Menikah'] = '1'
df.loc[(df['Menikah'].isna()) & (df['Jenis Kelamin'] == 'Pria') & (df['Umur'] < rerata_cwo), 'Menikah'] = '0'
df.loc[(df['Menikah'].isna()) & (df['Jenis Kelamin'] == 'Wanita') & (df['Umur'] >= rerata_cwe), 'Menikah'] = '1'
df.loc[(df['Menikah'].isna()) & (df['Jenis Kelamin'] == 'Wanita') & (df['Umur'] < rerata_cwe), 'Menikah'] = '0'
df.loc[df['Menikah'] == ' ', 'Menikah'] = '0'
df.loc[df['Promo'] > 3, 'Promo'] = 3
df.loc[df['Pendapatan-per-bulan'] == 0, 'Pendapatan-per-bulan'] = rerata_pendapatan

In [27]:
for kolom in df.columns:
    print(f'kolom: {kolom}\ntotal unique: {df[kolom].nunique()}\nunique: {df[kolom].unique()}\n')

kolom: Jenis Kelamin
total unique: 3
unique: ['Wanita', 'Pria', NaN, '-']
Categories (3, object): ['-', 'Pria', 'Wanita']

kolom: Umur
total unique: 43
unique: [49 32 51 19 38 33 27 24 21 31 36 46 60 58 47 45 22 18 59 44 39 35 20 30
 23 48 37 40 34 41 53 54 28 55 25 50 42 29 43 52 56 26 57]

kolom: Pendapatan-per-bulan
total unique: 64
unique: [13114754 33606557 15983606 12704918 20081967 29098360 21311475  2049180
 13524590 19262295 11475409 29508196 34016393  6147540 11885245 25819672
 25000000  5327868  9836065 23770491 22950819  1229508  9016393  9426229
 29918032 28688524 27049180 18442622 19672131 11065573 36065573 20491803
 17622950   409836 25409836  1639344  5737704 34426229 10245901 50000000
 14344262 35245901  7377049 45491803 40163934 17213114 23360655  7786885
 18852459 22131147  3688524  2459016  3278688 24180327 22540983   819672
 13934426 26229508 31967213 24590163 18032786 43032786 18675343  4098360]

kolom: skor-pengeluaran
total unique: 85
unique: [ 52  86  41  55  4

In [28]:
df = df[df['FavFlavor'] <= 8]
df.drop(df[df["Jenis Kelamin"] == '-'].index, inplace=True)
df.dropna(inplace=True)

In [29]:
for kolom in df.columns:
    print(f'kolom: {kolom}\ntotal unique: {df[kolom].nunique()}\nunique: {df[kolom].unique()}\n')

kolom: Jenis Kelamin
total unique: 2
unique: ['Wanita', 'Pria']
Categories (3, object): ['-', 'Pria', 'Wanita']

kolom: Umur
total unique: 43
unique: [49 32 51 19 38 33 27 24 21 36 46 60 58 47 45 22 18 59 44 39 35 20 30 23
 48 37 40 34 41 53 54 28 55 31 25 50 42 29 43 52 56 26 57]

kolom: Pendapatan-per-bulan
total unique: 64
unique: [13114754 33606557 15983606 12704918 20081967 29098360 21311475  2049180
 13524590 19262295 29508196 34016393  6147540 11885245 25819672 25000000
  5327868 11475409 23770491 22950819  1229508  9016393  9426229 29918032
 28688524 27049180 18442622 19672131 11065573 36065573 20491803 17622950
   409836 25409836  1639344  5737704 34426229  9836065 10245901 50000000
 14344262 35245901  7377049 45491803 40163934 17213114 23360655  7786885
 18852459 22131147  3688524  2459016  3278688 24180327 22540983 13934426
 26229508 31967213 24590163 18032786 43032786  4098360 18675343   819672]

kolom: skor-pengeluaran
total unique: 85
unique: [ 52  86  41  55  42  95  56 

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 187 entries, 0 to 199
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   Jenis Kelamin         187 non-null    category
 1   Umur                  187 non-null    int32   
 2   Pendapatan-per-bulan  187 non-null    int32   
 3   skor-pengeluaran      187 non-null    int64   
 4   Menikah               187 non-null    object  
 5   Kota                  187 non-null    object  
 6   Promo                 187 non-null    float64 
 7   HariFav               187 non-null    object  
 8   FavFlavor             187 non-null    int64   
dtypes: category(1), float64(1), int32(2), int64(2), object(3)
memory usage: 12.0+ KB


In [31]:
df.describe(include='all')

,Jenis Kelamin,Umur,Pendapatan-per-bulan,skor-pengeluaran,Menikah,Kota,Promo,HariFav,FavFlavor
count,187,187.000000,1.870000e+02,187.000000,187,187,187.000000,187,187.000000
unique,2,NaN,NaN,NaN,2,3,NaN,7,NaN
top,Wanita,NaN,NaN,NaN,1,Jakarta,NaN,Senin,NaN
freq,103,NaN,NaN,NaN,159,81,NaN,64,NaN
mean,NaN,37.631016,1.891945e+07,50.502674,NaN,NaN,1.652406,NaN,3.577540
std,NaN,11.905890,1.067615e+07,26.167035,NaN,NaN,0.945864,NaN,2.731365
min,NaN,18.000000,4.098360e+05,1.000000,NaN,NaN,0.000000,NaN,0.000000
25%,NaN,28.000000,1.147541e+07,34.500000,NaN,NaN,1.000000,NaN,1.000000
50%,NaN,36.000000,1.926230e+07,50.000000,NaN,NaN,1.000000,NaN,3.000000
75%,NaN,48.000000,2.581967e+07,73.000000,NaN,NaN,3.000000,NaN,6.000000


In [32]:
df['Menikah'] = df['Menikah'].astype(int)
df['Kota'] = df['Kota'].astype('category')
df['Promo'] = df['Promo'].astype(int)
df['HariFav'] = df['HariFav'].astype('category')

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 187 entries, 0 to 199
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   Jenis Kelamin         187 non-null    category
 1   Umur                  187 non-null    int32   
 2   Pendapatan-per-bulan  187 non-null    int32   
 3   skor-pengeluaran      187 non-null    int64   
 4   Menikah               187 non-null    int32   
 5   Kota                  187 non-null    category
 6   Promo                 187 non-null    int32   
 7   HariFav               187 non-null    category
 8   FavFlavor             187 non-null    int64   
dtypes: category(3), int32(4), int64(2)
memory usage: 8.5 KB


In [34]:
df.describe(include='all')

,Jenis Kelamin,Umur,Pendapatan-per-bulan,skor-pengeluaran,Menikah,Kota,Promo,HariFav,FavFlavor
count,187,187.000000,1.870000e+02,187.000000,187.000000,187,187.000000,187,187.000000
unique,2,NaN,NaN,NaN,NaN,3,NaN,7,NaN
top,Wanita,NaN,NaN,NaN,NaN,Jakarta,NaN,Senin,NaN
freq,103,NaN,NaN,NaN,NaN,81,NaN,64,NaN
mean,NaN,37.631016,1.891945e+07,50.502674,0.850267,NaN,1.652406,NaN,3.577540
std,NaN,11.905890,1.067615e+07,26.167035,0.357767,NaN,0.945864,NaN,2.731365
min,NaN,18.000000,4.098360e+05,1.000000,0.000000,NaN,0.000000,NaN,0.000000
25%,NaN,28.000000,1.147541e+07,34.500000,1.000000,NaN,1.000000,NaN,1.000000
50%,NaN,36.000000,1.926230e+07,50.000000,1.000000,NaN,1.000000,NaN,3.000000
75%,NaN,48.000000,2.581967e+07,73.000000,1.000000,NaN,3.000000,NaN,6.000000


In [35]:
df.sample(6)

,Jenis Kelamin,Umur,Pendapatan-per-bulan,skor-pengeluaran,Menikah,Kota,Promo,HariFav,FavFlavor
89,Pria,28,25409836,97,1,Depok,0,Senin,0
175,Pria,43,25819672,17,1,Tangerang,3,Senin,5
45,Pria,59,15983606,47,1,Jakarta,2,Minggu,8
21,Pria,18,13524590,59,0,Jakarta,1,Selasa,4
148,Wanita,21,7377049,81,0,Depok,1,Senin,8
35,Pria,19,27049180,5,0,Tangerang,3,Senin,5


In [36]:
def encode_kota(x):
    if x == "Jakarta":
        return 0
    if x == "Tangerang":
        return 1
    if x == "Depok":
        return 2
    
def encode_HariFav(x):
    hari = ['Minggu', 'Senin', 'Selasa', 'Rabu', 'Kamis', 'Jumat', 'Sabtu']
    return hari.index(x)

def encode_JK(x):
    if x == 'Pria':
        return 0
    if x == 'Wanita':
        return 1

In [37]:
df_encode = df[['Umur', 'Pendapatan-per-bulan', 'skor-pengeluaran', 'Menikah', 'Promo', 'FavFlavor']]
df_encode['kota encode'] = df['Kota'].apply(encode_kota)
df_encode['HariFav encode'] = df['HariFav'].apply(encode_HariFav)
df_encode['Kelamin encode'] = df['Jenis Kelamin'].apply(encode_JK)

C:\Users\Dell\AppData\Local\Temp\ipykernel_13284\223468718.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encode['kota encode'] = df['Kota'].apply(encode_kota)
C:\Users\Dell\AppData\Local\Temp\ipykernel_13284\223468718.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encode['HariFav encode'] = df['HariFav'].apply(encode_HariFav)


In [38]:
df_encode.head()

,Umur,Pendapatan-per-bulan,skor-pengeluaran,Menikah,Promo,FavFlavor,kota encode,HariFav encode,Kelamin encode
0,49,13114754,52,1,1,2,0,6,1.0
1,32,33606557,86,1,1,6,2,0,1.0
2,51,15983606,41,1,2,3,0,0,0.0
3,19,12704918,55,0,1,7,0,3,0.0
4,38,20081967,42,1,2,8,0,1,1.0


In [39]:
df_encode.info()

<class 'pandas.core.frame.DataFrame'>
Index: 187 entries, 0 to 199
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   Umur                  187 non-null    int32   
 1   Pendapatan-per-bulan  187 non-null    int32   
 2   skor-pengeluaran      187 non-null    int64   
 3   Menikah               187 non-null    int32   
 4   Promo                 187 non-null    int32   
 5   FavFlavor             187 non-null    int64   
 6   kota encode           187 non-null    category
 7   HariFav encode        187 non-null    category
 8   Kelamin encode        187 non-null    float64 
dtypes: category(2), float64(1), int32(4), int64(2)
memory usage: 9.6 KB


In [40]:
#df.to_csv("D:/DANIs D/KULIAH/Semester 5/Data Mining/Study Case/02/eskrim_PreProcessed.csv", encoding='utf8', index=False)
#df_encode.to_csv("D:/DANIs D/KULIAH/Semester 5/Data Mining/Study Case/02/eskrim_PreProcessed_encode.csv", encoding='utf8', index=False)
